<a href="https://colab.research.google.com/github/meetgandhi123/PyTorch-Basic-Concepts/blob/main/CIFAR_10_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transform
import matplotlib.pyplot as plt
import numpy as np

# Device Config
Device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device usage: {Device}')

# Hyper Parameters.
num_epoch = 20
batch_size = 4 
learning_rate = 0.001 

# Applying Transforms
transform = transform.Compose([transform.ToTensor(), 
                               transform.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
                              ])

# Dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download = True, transform=transform) 
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download = False, transform=transform)

# Dataloaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle = False)

# Classes
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'forg' 'horse', 'ship', 'truck' )

# Model
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1,16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = ConvNet().to(Device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

n_total_steps = len(train_loader)
for epoch in range(num_epoch):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(Device)
        labels = labels.to(Device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epoch}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finish Training.')

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(Device)
        labels = labels.to(Device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

Device usage: cuda
Files already downloaded and verified
Epoch [1/20], Step [2000/12500], Loss: 2.3244
Epoch [1/20], Step [4000/12500], Loss: 2.3123
Epoch [1/20], Step [6000/12500], Loss: 2.2848
Epoch [1/20], Step [8000/12500], Loss: 2.3098
Epoch [1/20], Step [10000/12500], Loss: 2.3131
Epoch [1/20], Step [12000/12500], Loss: 2.2060
Epoch [2/20], Step [2000/12500], Loss: 2.2076
Epoch [2/20], Step [4000/12500], Loss: 3.8316
Epoch [2/20], Step [6000/12500], Loss: 2.1280
Epoch [2/20], Step [8000/12500], Loss: 1.5550
Epoch [2/20], Step [10000/12500], Loss: 1.6399
Epoch [2/20], Step [12000/12500], Loss: 2.3553
Epoch [3/20], Step [2000/12500], Loss: 2.1220
Epoch [3/20], Step [4000/12500], Loss: 2.1125
Epoch [3/20], Step [6000/12500], Loss: 2.5035
Epoch [3/20], Step [8000/12500], Loss: 1.5516
Epoch [3/20], Step [10000/12500], Loss: 1.1157
Epoch [3/20], Step [12000/12500], Loss: 1.8685
Epoch [4/20], Step [2000/12500], Loss: 1.2142
Epoch [4/20], Step [4000/12500], Loss: 1.4171
Epoch [4/20], Ste